In [ ]:
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs.play import play
import os

load_dotenv()
#tts
elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

audio = elevenlabs.text_to_speech.convert(
    text="The first move is what sets everything in motion.",
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

play(audio)



In [ ]:
import requests

r = requests.post("http://localhost:8000/call-me")
r.json()

In [ ]:
import json, websockets

async def listen():
    uri = "ws://localhost:8000/ws/client"
    async with websockets.connect(uri) as ws:
        print("Connected to /ws/client (transcript stream)")
        # send anything occasionally so the server's ws_client loop doesn't block forever
        await ws.send("hi")
        while True:
            raw = await ws.recv()
            msg = json.loads(raw)
            src = msg.get("source")
            ev = msg.get("event")

            if src == "eleven" and ev == "transcript":
                    tag = "FINAL" if msg.get("message_type") == 'committed_transcript' else "PARTIAL"
                    print(f"{tag}: {msg.get('text')}")
            elif ev in {"error", "closed"}:
                    print(msg)

await listen()


Connected to /ws/client (transcript stream)
{'source': 'eleven', 'event': 'message', 'call_sid': 'CAb875b2360c68f799f8becca842ee11a4', 'stream_sid': 'MZ11a144a41ac5817dcc148944af146a7c', 'raw': {'message_type': 'committed_transcript', 'text': ''}}
{'source': 'eleven', 'event': 'message', 'call_sid': 'CAb875b2360c68f799f8becca842ee11a4', 'stream_sid': 'MZ11a144a41ac5817dcc148944af146a7c', 'raw': {'message_type': 'committed_transcript', 'text': ''}}
{'source': 'eleven', 'event': 'transcript', 'call_sid': 'CAb875b2360c68f799f8becca842ee11a4', 'stream_sid': 'MZ11a144a41ac5817dcc148944af146a7c', 'text': 'all right', 'is_final': False, 'raw': {'message_type': 'partial_transcript', 'text': 'all right'}}
{'source': 'eleven', 'event': 'message', 'call_sid': 'CAb875b2360c68f799f8becca842ee11a4', 'stream_sid': 'MZ11a144a41ac5817dcc148944af146a7c', 'raw': {'message_type': 'committed_transcript', 'text': ''}}
{'source': 'eleven', 'event': 'transcript', 'call_sid': 'CAb875b2360c68f799f8becca842ee11a

CancelledError: 

In [7]:
import requests

r = requests.post("http://localhost:8000/call-me")
r.json()

{'status': 'calling', 'sid': 'CAb875b2360c68f799f8becca842ee11a4'}